In [12]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from numba import cuda
import math
import pickle
from annoy import AnnoyIndex
from tqdm import tqdm

patient_icd_path="/data1/andrew/meng/mixehr/data/Mimic/andrew_outputs/PATIENT_ICD_BINARY.csv"

In [3]:
patient_icd_df = pd.read_csv(patient_icd_path, sep=' ')
patient_icd_df

,SUBJECT_ID,0030,0031,0038,0039,0041,0048,0049,0051,00581,...,V8801,V8811,V8812,V8821,V9010,V902,V9039,V9081,V9089,V9103
0,109,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,112,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,113,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,114,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,115,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,116,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,117,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,118,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,119,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,120,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
icd_code_idxs = patient_icd_df.drop("SUBJECT_ID", axis=1).columns
patient_idxs = patient_icd_df.index

In [5]:
patient_data = patient_icd_df.drop('SUBJECT_ID', axis=1).as_matrix().astype(np.float)
print(type(patient_data))

/data1/andrew/anaconda2/envs/mengp369/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


<class 'numpy.ndarray'>


In [1]:
sim_matrix = pickle.load(open("euc_sim_matrix.p", "rb"))
euc_sim_df = pd.DataFrame(sim_matrix, index=patient_idxs)
euc_sim_df.insert(loc=0, column="SUBJECT_ID", value=patient_icd_df["SUBJECT_ID"])
euc_sim_df

NameError: name 'pickle' is not defined

In [13]:
f = patient_data.shape[1]
t = AnnoyIndex(f, "angular")
for i, row in tqdm(enumerate(patient_data)):
    t.add_item(i, row)
t.build(10)

46520it [00:42, 1086.85it/s]


True

In [14]:
t.save("t_idx.ann")

True

In [15]:
t.get_nns_by_item(0, 100)

[0,
 3319,
 29662,
 24132,
 25275,
 16666,
 37755,
 39245,
 16376,
 14844,
 26326,
 34948,
 43782,
 37026,
 41266,
 34699,
 575,
 7206,
 16593,
 44554,
 26000,
 27994,
 27511,
 19853,
 763,
 6929,
 8225,
 37687,
 29390,
 26986,
 11416,
 20125,
 42398,
 5053,
 35708,
 14472,
 25143,
 28145,
 5676,
 1949,
 7456,
 11647,
 27122,
 12725,
 28544,
 33250,
 13828,
 16693,
 22743,
 3149,
 11291,
 10888,
 35543,
 34792,
 36782,
 20060,
 18264,
 7334,
 2434,
 8952,
 21436,
 1561,
 13602,
 8564,
 46253,
 44929,
 23160,
 3045,
 40991,
 46441,
 29192,
 17348,
 32539,
 34803,
 38108,
 25299,
 40372,
 6925,
 28431,
 9246,
 21353,
 26241,
 46116,
 37539,
 25860,
 33335,
 6383,
 20841,
 31305,
 42450,
 44211,
 34423,
 35419,
 44163,
 437,
 19603,
 26430,
 33546,
 36503,
 1319]

In [ ]:
@cuda.jit
def euc_sim_matrix(A, B, C):
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        sqdist = 0
        for i in range(A.shape[1]):
            sqdist += (A[row, i] - B[i, col])**2
        euc_sim = 1 / (1 + math.sqrt(sqdist)) #Caps similarity at 1 and avoids division by 0
        C[row][col] = euc_sim

chunksize = 10000
endidx = chunksize
sim_matrix=np.empty(shape=(0, patient_data.shape[0]))

while True:        
    print("Current End Index: {} with chunk size of {}".format(endidx, chunksize))
    endidx = min(patient_data.shape[0], endidx)
    A = np.ascontiguousarray(patient_data[:endidx])
    B = patient_data.T
    print(A.shape, B.shape)
    print(type(A[0]), type(B[0]))

    A_mem = cuda.to_device(A)
    B_mem = cuda.to_device(B)
    C_mem = cuda.device_array((A.shape[0], B.shape[1]), dtype=np.float)

    threadsperblock=(16, 16)
    blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[0]))
    blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[1]))
    blockspergrid=(blockspergrid_x, blockspergrid_y)

    euc_sim_matrix[blockspergrid, threadsperblock](A_mem, B_mem, C_mem)
    C = C_mem.copy_to_host()
    np.append(sim_matrix, C, axis=0)
    print(C.shape)
    cuda.current_context().reset()
    if endidx == patient_data.shape[0]:
        break
    endidx += chunksize


Current End Index: 10000 with chunk size of 10000
(10000, 6984) (6984, 46520)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(10000, 46520)
Current End Index: 20000 with chunk size of 10000
(20000, 6984) (6984, 46520)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [ ]:
sim_matrix